### Initial test

In [ ]:
from pde import DiffusionPDE, ScalarField, UnitGrid

grid = UnitGrid([64, 64])  # generate grid
state = ScalarField.random_uniform(grid, 0.2, 0.3)  # generate initial condition

eq = DiffusionPDE(diffusivity=0.1)  # define the pde
result = eq.solve(state, t_range=10)
result.plot()

## Turing-like R/A system

Initilize fields

In [ ]:
# from pde import CartesianGrid, ScalarField, PDE
# import numpy as np
# import matplotlib.pyplot as plt

# # (i) Define a 2D periodic grid
# L = 50.0                     # physical length
# N = 128                      # number of grid points per axis
# grid = CartesianGrid([[0, L], [0, L]], shape=(N, N), periodic=True)

# # (ii) Initialize fields directly from NumPy arrays
# D_data = 1 + 0.05 * np.sin(2 * np.pi * np.linspace(0, 1, N))[:, None] * np.ones((1, N))
# A_data = 0.5 + 0.1 * np.random.rand(N, N)
# R_data = 0.5 + 0.1 * np.random.rand(N, N)

# D = ScalarField(grid, data=D_data)
# A = ScalarField(grid, data=A_data)
# R = ScalarField(grid, data=R_data)

# # Optional visualization
# fig, axs = plt.subplots(1, 3, figsize=(12, 4))
# D.plot(ax=axs[0], title="Substrate D")
# A.plot(ax=axs[1], title="Activator A")
# R.plot(ax=axs[2], title="Repressor R")
# plt.tight_layout(); plt.show()

Define a function that gives the logic of the PDE system

In [ ]:
from pde import FieldCollection, PDEBase, PlotTracker, ScalarField, UnitGrid


class SIRPDE(PDEBase):
    """SIR-model with diffusive mobility"""

    def __init__(
        self, beta=0.3, gamma=0.9, diffusivity=0.1, bc="auto_periodic_neumann"
    ):
        super().__init__()
        self.beta = beta  # transmission rate
        self.gamma = gamma  # recovery rate
        self.diffusivity = diffusivity  # spatial mobility
        self.bc = bc  # boundary condition

    def get_state(self, s, i):
        """generate a suitable initial state"""
        norm = (s + i).data.max()  # maximal density
        if norm > 1:
            s /= norm
            i /= norm
        s.label = "Susceptible"
        i.label = "Infected"

        # create recovered field
        r = ScalarField(s.grid, data=1 - s - i, label="Recovered")
        return FieldCollection([s, i, r])

    def evolution_rate(self, state, t=0):
        s, i, r = state
        diff = self.diffusivity
        ds_dt = diff * s.laplace(self.bc) - self.beta * i * s
        di_dt = diff * i.laplace(self.bc) + self.beta * i * s - self.gamma * i
        dr_dt = diff * r.laplace(self.bc) + self.gamma * i
        return FieldCollection([ds_dt, di_dt, dr_dt])


eq = SIRPDE(beta=2, gamma=0.1)

# initialize state
grid = UnitGrid([32, 32])
s = ScalarField(grid.copy(), 1)
i = ScalarField(grid.copy(), 0)
i.data[0, 0] = 1
state = eq.get_state(s, i)

# simulate the pde
tracker = PlotTracker(interval=10, plot_args={"vmin": 0, "vmax": 1})
sol = eq.solve(state, t_range=50, dt=1e-2, tracker=["progress", tracker])

In [ ]:
from pde import ScalarField, FieldCollection, PDEBase
# from pde.tools.misc import skip_args

class TuringPDE(PDEBase):
    def __init__(self,
                 Da=0.01, Dr=0.2,
                 alpha=1.0, beta=1.0, gamma=0.5, decay=0.1,
                 bc="auto_periodic_neumann",
                 n_A_spots=5, n_R_spots=2,hill=None):
        super().__init__()
        self.Da = Da
        self.Dr = Dr
        self.alpha = alpha
        self.beta = beta
        self.gamma = gamma
        self.decay = decay
        self.bc = bc
        self.n_A_spots = n_A_spots
        self.n_R_spots = n_R_spots
        self.hill = hill

    def evolution_rate(self, state, t=0):
        A, R = state  # Activator, Repressor

        f_A = self.alpha * A**self.hill0 / (1 + R**self.hill1) - self.beta * A
        f_R = self.gamma * A**self.hill2 - self.decay * R

        dA_dt = self.Da * A.laplace(self.bc) + f_A
        dR_dt = self.Dr * R.laplace(self.bc) + f_R

        return FieldCollection([dA_dt, dR_dt])

    def get_state(self, grid):
        A = self.make_spot_field(grid, n_spots=self.n_A_spots,
                                 radius=2, amplitude_range=(0.5, 1.0)).copy(label="Activator")
        R = self.make_spot_field(grid, n_spots=self.n_R_spots,
                                 radius=2, amplitude_range=(0.2, 0.6)).copy(label="Repressor")
        return FieldCollection([A, R])

    def make_spot_field(self, grid, n_spots=3, radius=2, amplitude_range=(0.5, 1.0), seed=None):
        if seed is not None:
            np.random.seed(seed)
    
        field = ScalarField(grid, data=0.0)
    
        # Build coordinate arrays manually
        Nx, Ny = grid.shape
        (x_min, x_max), (y_min, y_max) = grid.axes_bounds
        x = np.linspace(x_min, x_max, Nx)
        y = np.linspace(y_min, y_max, Ny)
        X, Y = np.meshgrid(x, y, indexing="ij")  # shape (Nx, Ny)
    
        for _ in range(n_spots):
            cx, cy = np.random.uniform(x_min, x_max), np.random.uniform(y_min, y_max)
            # radius = np.random.uniform(*radius_range)
            amplitude = np.random.uniform(*amplitude_range)
    
            dist2 = (X - cx)**2 + (Y - cy)**2
            field.data += amplitude * np.exp(-dist2 / (2 * radius**2))
    
        return field
    


In [ ]:
from pde import UnitGrid, PlotTracker

from pde import CartesianGrid
L = 50.0  # Physical domain size
N = 128   # Grid resolution
grid = CartesianGrid([[0, L], [0, L]], shape=(N, N), periodic=True)

# Initialize PDE and state
eq = TuringPDE(
    Da=0.005, Dr=0.3,
    alpha=2.0,   # ↑ stronger A production
    beta=0.002,    # ↓ slower A degradation
    gamma=10,   # ↑ stronger R production
    decay=0.002,    # ↓ slower R degradation,
    n_A_spots=2,
    n_R_spots=0,
    hill=5
)
state = eq.get_state(grid)

# Run simulation with inline visualization
tracker = PlotTracker(interval=100)
result = eq.solve(state, t_range=2500, dt=0.1, tracker=["progress", tracker])

In [ ]:
12*4*3*3